# Identifying and filling in missing data

In [1]:
import pandas as pd
import numpy as np

In [2]:
titanic_df = pd.read_csv('../datasets/titanic/train.csv')

In [3]:
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### How many values of Age are missing?

In [4]:
sum(titanic_df['Age'].isnull() == True)

177

In [5]:
desc = titanic_df['Age'].describe()
desc

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: Age, dtype: float64

In [6]:
from scipy import stats

most_freq_age, count = stats.mode(titanic_df['Age'])

age_mode = int(most_freq_age)
titanic_df['Age_repl1'] = titanic_df['Age']
titanic_df['Age_repl1'].replace(np.nan, age_mode, inplace=True)
desc1 = titanic_df['Age_repl1'].describe()
desc1

count    891.000000
mean      28.566970
std       13.199572
min        0.420000
25%       22.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: Age_repl1, dtype: float64

# Solution

## 1. Using Constant Value (0)
* Replacing missing values with contant value 0.

In [7]:
titanic_df["age_repl2"] = titanic_df["Age"]
titanic_df["age_repl2"].replace(np.nan, 0, inplace=True)
print("Any nulls are there?",sum(titanic_df['age_repl2'].isnull() == True)) #just to validate if there are not null values in age_repl2 column
desc2 = titanic_df["age_repl2"].describe()
desc2

Any nulls are there? 0


count    891.000000
mean      23.799293
std       17.596074
min        0.000000
25%        6.000000
50%       24.000000
75%       35.000000
max       80.000000
Name: age_repl2, dtype: float64

## 2. Using median 
* Replacing the missing values using column median value

In [8]:
titanic_df["age_repl3"] = titanic_df["Age"]
titanic_df["age_repl3"].fillna(titanic_df["age_repl3"].median(), inplace=True)
print("Any nulls are there?",sum(titanic_df['age_repl3'].isnull() == True))
desc3 = titanic_df["age_repl3"].describe()
desc3

Any nulls are there? 0


count    891.000000
mean      29.361582
std       13.019697
min        0.420000
25%       22.000000
50%       28.000000
75%       35.000000
max       80.000000
Name: age_repl3, dtype: float64

## 3. Using values from previous row

In [9]:
titanic_df["age_repl4"] = titanic_df["Age"]
titanic_df["age_repl4"].fillna(method='pad', inplace=True)
print("Any nulls are there?",sum(titanic_df['age_repl4'].isnull() == True))
desc4 = titanic_df["age_repl4"].describe()
desc4 

Any nulls are there? 0


count    891.00000
mean      29.58156
std       14.55459
min        0.42000
25%       20.00000
50%       28.00000
75%       38.00000
max       80.00000
Name: age_repl4, dtype: float64

## 4. Using values from next row
* Replacing value with the next row


In [10]:
titanic_df["age_repl5"] = titanic_df["Age"]
titanic_df["age_repl5"].fillna(method='bfill', inplace=True)
print("Any nulls are there?",sum(titanic_df['age_repl5'].isnull() == True))
desc5 = titanic_df["age_repl5"].describe()
desc5

Any nulls are there? 0


count    891.000000
mean      29.870561
std       14.597668
min        0.420000
25%       21.000000
50%       29.000000
75%       39.000000
max       80.000000
Name: age_repl5, dtype: float64

## 5. Using Interpolate Method
* Replacing values using linear interpolate methid

In [11]:
titanic_df["age_repl6"] = titanic_df["Age"]
titanic_df["age_repl6"].interpolate(method='linear', direction = 'forward', inplace=True) 
print("Any nulls are there?",sum(titanic_df['age_repl6'].isnull() == True))
desc6 = titanic_df["age_repl6"].describe()
desc6

Any nulls are there? 0


count    891.000000
mean      29.726061
std       13.902353
min        0.420000
25%       21.000000
50%       28.500000
75%       38.000000
max       80.000000
Name: age_repl6, dtype: float64

### 6. Using other columns to group by and then impute missing value
* Replacing the missing values using the "sex" column and group by as females and males to impute missing values in a group

In [12]:
sum(titanic_df["Sex"].isnull()==True)

0

In [13]:
df1 = pd.DataFrame(columns=["age_repl7","sex"])
df1["age_repl7"]=titanic_df["Age"]
df1["sex"] = titanic_df["Sex"]
df1.head()

,age_repl7,sex
0,22.0,male
1,38.0,female
2,26.0,female
3,35.0,female
4,35.0,male


In [14]:
df1["age_repl7"] = df1.groupby("sex")["age_repl7"].transform(lambda x: x.fillna(x.mean()))
print("Any nulls are there?",sum(df1['age_repl7'].isnull() == True))
desc7 = df1["age_repl7"].describe()
desc7

Any nulls are there? 0


count    891.000000
mean      29.736034
std       13.014897
min        0.420000
25%       22.000000
50%       30.000000
75%       35.000000
max       80.000000
Name: age_repl7, dtype: float64

### 7. Using mean of age attribute

In [15]:
titanic_df["age_repl8"] = titanic_df["Age"]
titanic_df["age_repl8"].fillna(titanic_df["age_repl8"].mean(), inplace=True)
print("Any nulls are there?",sum(titanic_df['age_repl8'].isnull() == True))
desc8 = titanic_df["age_repl8"].describe()
desc8

Any nulls are there? 0


count    891.000000
mean      29.699118
std       13.002015
min        0.420000
25%       22.000000
50%       29.699118
75%       35.000000
max       80.000000
Name: age_repl8, dtype: float64

### 8. Using Standard Deviation

In [16]:
titanic_df["age_repl9"] = titanic_df["Age"]
titanic_df["age_repl9"].fillna(titanic_df["age_repl9"].std(), inplace=True)
print("Any nulls are there?",sum(titanic_df['age_repl9'].isnull() == True))
desc9 = titanic_df["age_repl9"].describe()
desc9

Any nulls are there? 0


count    891.000000
mean      26.685028
std       14.343656
min        0.420000
25%       14.526497
50%       24.000000
75%       35.000000
max       80.000000
Name: age_repl9, dtype: float64

In [17]:
df_age = pd.DataFrame(desc, columns=["Age"])
df_mode = pd.DataFrame(desc1, columns=["age_repl1"])
df_const = pd.DataFrame(desc2, columns=["age_repl2"])
df_med= pd.DataFrame(desc3, columns=["age_repl3"])
df_prev = pd.DataFrame(desc4, columns=["age_repl4"])
df_next = pd.DataFrame(desc5, columns=["age_repl5"])
df_inter = pd.DataFrame(desc6, columns=["age_repl6"])
df_grp = pd.DataFrame(desc7, columns=["age_repl7"])
df_mean = pd.DataFrame(desc8, columns=["age_repl8"])
df_std = pd.DataFrame(desc9, columns=["age_repl9"])


In [18]:
final_result = pd.concat([df_age,df_const,df_med,df_prev,df_next,df_inter,df_grp,df_mean,df_std],axis=1)
final_result.rename(columns={'age_repl1':'Mode','age_repl2':'Const(0)','age_repl3':'Median','age_repl4':'PrevVal',
                            'age_repl5':'NextVal','age_repl6':'Interpo','age_repl7':'GroupBy','age_repl8':'mean',
                            'age_repl9':'Std'}, inplace=True)
            


### Analysis Results
* If we look at the table, we can clearly see that filling missing values using `Next available value in the dataset, Previous available values, and Linear Interpolation` are likely to have closest statistic properties as of original Age attribute.
* Using constant value which Zero, the statistic properties are deviant to the origanl ones. So it is not a good option to use constant value for missing data as it can create biasness in the dataset.


In [19]:
final_result.style.apply(lambda x: ['background:lightgreen' if x=='Age' else '' 
                                    'background:lightgreen' if x=='PrevVal' or x=='NextVal'
                                    or x=='Interpo' else '' for x in final_result.columns], axis=1)

,Age,Const(0),Median,PrevVal,NextVal,Interpo,GroupBy,mean,Std
count,714.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,29.699118,23.799293,29.361582,29.581560,29.870561,29.726061,29.736034,29.699118,26.685028
std,14.526497,17.596074,13.019697,14.554590,14.597668,13.902353,13.014897,13.002015,14.343656
min,0.420000,0.000000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000,0.420000
25%,20.125000,6.000000,22.000000,20.000000,21.000000,21.000000,22.000000,22.000000,14.526497
50%,28.000000,24.000000,28.000000,28.000000,29.000000,28.500000,30.000000,29.699118,24.000000
75%,38.000000,35.000000,35.000000,38.000000,39.000000,38.000000,35.000000,35.000000,35.000000
max,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000


## Part2
### Taking Embarked and Cabin properties of passenger and Using PAD(Previous Value) method as replacement strategy for missing values

In [20]:
df_pType = titanic_df[["Embarked","Cabin"]] 
df_pType.head()

,Embarked,Cabin
0,S,NaN
1,C,C85
2,S,NaN
3,S,C123
4,S,NaN


In [21]:
desc_emb = df_pType["Embarked"].describe()
desc_sur = df_pType["Cabin"].describe()

df_emb = pd.DataFrame(desc_emb, columns=["Embarked"])
df_sur = pd.DataFrame(desc_sur, columns=["cabin"])

final_pType = pd.concat([desc_emb,desc_sur],axis=1)
final_pType

,Embarked,Cabin
count,889,204
unique,3,147
top,S,G6
freq,644,4


In [28]:
df_pType["Embarked"].fillna(method='pad', inplace=True)
df_pType["Cabin"].fillna(method='pad', inplace=True)
desc_emb_repl = df_pType["Embarked"].describe()
desc_sur_repl = df_pType["Cabin"].describe()
df_emb_repl = pd.DataFrame(desc_emb_repl, columns=["Embarked"])
df_sur_repl = pd.DataFrame(desc_sur_repl, columns=["Cabin"])
final_pType_repl = pd.concat([df_emb_repl,df_sur_repl],axis=1)
final_pType_repl.rename(columns={"Embarked":"Embarked_repl","Cabin":"Cabin_repl",
                           },inplace=True)
final_pType_repl

/usr/local/lib/python3.7/site-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,Embarked_repl,Cabin_repl
count,891,891
unique,3,147
top,S,G6
freq,644,24


In [23]:
final_merged_df = final_pType.join(final_pType_repl)
final_merged_df.style.apply(lambda x: ['background:lightgreen' if x =='Embarked'or x=='Embarked_repl' else '' 
                                    'background:red' if x =='Cabin' or x=='Cabin_repl'
                                    else '' for x in final_merged_df.columns], axis=1)


,Embarked,Cabin,Embarked_repl,Cabin_repl
count,889,204,891,890
unique,3,147,3,147
top,S,G6,S,G6
freq,644,4,644,24


### Findings:
As clearly seen from the above table, the Embarked attribute is not affected by the replacement strategy used. However the Cabin's Freq property has been affected with the replacement startegy. 